# Load Necessary packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
import os
import pickle as pkl
# loading DESEQ in particular 
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

### Load in the new datasets

In [28]:
SAVE = True  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be saved
    OUTPUT_PATH = "/Users/shrinivas/Desktop/Sargassum/Working"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

In [16]:
counts_df = pd.read_csv(r'/Users/shrinivas/Desktop/Sargassum/Working/DESEQ_input.csv', index_col='Sample')
metadata = pd.read_csv(r'/Users/shrinivas/Desktop/Sargassum/Working/LCC6_Metadata.csv', index_col='Sample')
metadata

,Strain,Growth substrate,Timepoint [h],Sample accession,Identifier,Treatment
Sample,,,,,,
ERR3606203,Lentimonas' sp. CC6,iota-carrageenan (Sigma-Aldrich C1138),40,SAMEA6101969,LCC6_iota_carrageenan_40h,iota_carrageenan_40h
ERR3606204,Lentimonas' sp. CC6,iota-carrageenan (Sigma-Aldrich C1138),20,SAMEA6101970,LCC6_iota_carrageenan_20h,iota_carrageenan_20h
ERR3606205,Lentimonas' sp. CC6,Pure fucoidan from F. vesiculosus (Sigma-Aldri...,20,SAMEA6101971,LCC6_pure_fucoidan_20h,Pure_fucoidan_20h
ERR3606206,Lentimonas' sp. CC6,Pure fucoidan from F. vesiculosus (Sigma-Aldri...,40,SAMEA6101972,LCC6_pure_fucoidan_40h,Pure_fucoidan_40h
ERR3606207,Lentimonas' sp. CC6,Crude fucoidan from F. vesiculosus (Sigma-Ald...,40,SAMEA6101973,LCC6_crude_fucoidan_40h,Crude_fucoidan_40h
ERR3606208,Lentimonas' sp. CC6,Crude fucoidan from F. vesiculosus (Sigma-Ald...,20,SAMEA6101974,LCC6_crude_fucoidan_20h,Crude_fucoidan_20h
ERR3606209,Lentimonas' sp. CC6,Mannose (Sigma-Aldrich M6020),40,SAMEA6101975,LCC6_mannose_40h,Mannose_40h
ERR3606210,Lentimonas' sp. CC6,Mannose (Sigma-Aldrich M6020),20,SAMEA6101976,LCC6_mannose_20h,Mannose_20h


# Pre-processing

In [19]:
transposed_data = counts_df.T
transposed_data

Sample,lcl|CACVAH010000001.1_cds_CAA6683422.1_1,lcl|CACVAH010000001.1_cds_CAA6683423.1_2,lcl|CACVAH010000001.1_cds_CAA6683424.1_3,lcl|CACVAH010000001.1_cds_CAA6683425.1_4,lcl|CACVAH010000001.1_cds_CAA6683426.1_5,lcl|CACVAH010000001.1_cds_CAA6683427.1_6,lcl|CACVAH010000001.1_cds_CAA6683428.1_7,lcl|CACVAH010000001.1_cds_CAA6683429.1_8,lcl|CACVAH010000001.1_cds_CAA6683430.1_9,lcl|CACVAH010000001.1_cds_CAA6683431.1_10,...,lcl|CACVAH010000092.1_cds_CAA6687583.1_4084,lcl|CACVAH010000092.1_cds_CAA6687584.1_4085,lcl|CACVAH010000093.1_cds_CAA6687585.1_4086,lcl|CACVAH010000094.1_cds_CAA6687586.1_4087,lcl|CACVAH010000094.1_cds_CAA6687587.1_4088,lcl|CACVAH010000095.1_cds_CAA6687588.1_4089,lcl|CACVAH010000096.1_cds_CAA6687590.1_4090,lcl|CACVAH010000097.1_cds_CAA6687591.1_4091,lcl|CACVAH010000098.1_cds_CAA6687592.1_4092,lcl|CACVAH010000099.1_cds_CAA6687593.1_4093
ERR3606207,283.0,4266.0,2165.0,485.0,263.0,232.0,253.0,437.0,198.0,1318.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606210,215.0,3315.0,1963.0,1124.0,986.0,750.0,1059.0,1310.0,455.0,2087.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606205,282.0,4166.0,2793.0,1459.0,1206.0,984.0,1447.0,1746.0,610.0,2960.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606206,246.0,5219.0,2602.0,781.0,411.0,504.0,359.0,420.0,186.0,1143.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606204,268.0,1806.0,1093.0,501.0,430.0,353.0,475.0,1029.0,425.0,1972.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606203,188.0,1616.0,1012.0,220.0,152.0,133.0,108.0,242.0,155.0,842.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606209,293.0,6278.0,3385.0,1125.0,889.0,740.0,837.0,800.0,274.0,1768.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3606208,315.0,5456.0,3410.0,1396.0,1269.0,956.0,1312.0,1530.0,651.0,2989.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
genes_to_keep = transposed_data.columns[transposed_data.sum(axis=0) >= 10]
transposed_data = transposed_data[genes_to_keep]
transposed_data

Sample,lcl|CACVAH010000001.1_cds_CAA6683422.1_1,lcl|CACVAH010000001.1_cds_CAA6683423.1_2,lcl|CACVAH010000001.1_cds_CAA6683424.1_3,lcl|CACVAH010000001.1_cds_CAA6683425.1_4,lcl|CACVAH010000001.1_cds_CAA6683426.1_5,lcl|CACVAH010000001.1_cds_CAA6683427.1_6,lcl|CACVAH010000001.1_cds_CAA6683428.1_7,lcl|CACVAH010000001.1_cds_CAA6683429.1_8,lcl|CACVAH010000001.1_cds_CAA6683430.1_9,lcl|CACVAH010000001.1_cds_CAA6683431.1_10,...,lcl|CACVAH010000059.1_cds_CAA6687537.1_4038,lcl|CACVAH010000060.1_cds_CAA6687538.1_4039,lcl|CACVAH010000061.1_cds_CAA6687539.1_4040,lcl|CACVAH010000063.1_cds_CAA6687542.1_4043,lcl|CACVAH010000063.1_cds_CAA6687543.1_4044,lcl|CACVAH010000064.1_cds_CAA6687544.1_4045,lcl|CACVAH010000064.1_cds_CAA6687545.1_4046,lcl|CACVAH010000064.1_cds_CAA6687546.1_4047,lcl|CACVAH010000065.1_cds_CAA6687547.1_4048,lcl|CACVAH010000066.1_cds_CAA6687548.1_4049
ERR3606207,283.0,4266.0,2165.0,485.0,263.0,232.0,253.0,437.0,198.0,1318.0,...,19.888,95.0,13.0,171.0,21.0,5396.0,5094.156,10428.798,10.243,165.0
ERR3606210,215.0,3315.0,1963.0,1124.0,986.0,750.0,1059.0,1310.0,455.0,2087.0,...,55.559,13.0,6.0,137.0,14.0,2602.0,549.000,1176.531,5.970,96.0
ERR3606205,282.0,4166.0,2793.0,1459.0,1206.0,984.0,1447.0,1746.0,610.0,2960.0,...,33.314,10.0,10.0,142.0,12.0,3243.0,544.000,1099.906,3.471,98.0
ERR3606206,246.0,5219.0,2602.0,781.0,411.0,504.0,359.0,420.0,186.0,1143.0,...,32.597,53.0,8.0,137.0,18.0,4251.0,4620.358,9430.071,9.197,180.0
ERR3606204,268.0,1806.0,1093.0,501.0,430.0,353.0,475.0,1029.0,425.0,1972.0,...,1921.603,11.0,4.0,80.0,8.0,250.0,36.000,89.523,0.000,66.0
ERR3606203,188.0,1616.0,1012.0,220.0,152.0,133.0,108.0,242.0,155.0,842.0,...,94.616,115.0,10.0,82.0,9.0,1535.0,3004.176,8701.090,3.315,148.0
ERR3606209,293.0,6278.0,3385.0,1125.0,889.0,740.0,837.0,800.0,274.0,1768.0,...,43.400,32.0,4.0,168.0,26.0,6242.0,2509.673,6189.174,2.287,168.0
ERR3606208,315.0,5456.0,3410.0,1396.0,1269.0,956.0,1312.0,1530.0,651.0,2989.0,...,59.806,12.0,17.0,153.0,26.0,4157.0,1160.342,2805.465,1.085,155.0


In [21]:
# convert counts table to Integers
transposed_data = transposed_data.fillna(0).astype(int).astype(float)

In [22]:
# order the index correctly
print(transposed_data.index)
print(metadata.index)

# Sort indices in transposed_data
transposed_data_sorted = transposed_data.sort_index()

# Sort indices in metadata
metadata_sorted = metadata.sort_index()

# Print sorted indices
print(transposed_data_sorted.index)
print(metadata_sorted.index)

Index(['ERR3606207', 'ERR3606210', 'ERR3606205', 'ERR3606206', 'ERR3606204',
       'ERR3606203', 'ERR3606209', 'ERR3606208'],
      dtype='object')
Index(['ERR3606203', 'ERR3606204', 'ERR3606205', 'ERR3606206', 'ERR3606207',
       'ERR3606208', 'ERR3606209', 'ERR3606210'],
      dtype='object', name='Sample')
Index(['ERR3606203', 'ERR3606204', 'ERR3606205', 'ERR3606206', 'ERR3606207',
       'ERR3606208', 'ERR3606209', 'ERR3606210'],
      dtype='object')
Index(['ERR3606203', 'ERR3606204', 'ERR3606205', 'ERR3606206', 'ERR3606207',
       'ERR3606208', 'ERR3606209', 'ERR3606210'],
      dtype='object', name='Sample')


In [7]:
# Actually run DESEQ2
dds = DeseqDataSet(
    counts=transposed_data_sorted,
    metadata=metadata_sorted,
    design_factors=["Growth substrate"],
    refit_cooks=True
)

In [29]:
dds.deseq2()
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds.pkl"), "wb") as f:
        pkl.dump(dds, f)

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.71 seconds.

Fitting dispersion trend curve...
... done in 0.10 seconds.

Fitting MAP dispersions...
... done in 0.81 seconds.

Fitting LFCs...
... done in 0.43 seconds.

Replacing 0 outlier genes.



In [24]:
print(dds)


AnnData object with n_obs × n_vars = 8 × 4092
    obs: 'Strain', 'Growth substrate', 'Timepoint [h]', 'Sample accession', 'Identifier', 'Treatment'
    uns: 'trend_coeffs', 'disp_function_type', '_squared_logres', 'prior_disp_var'
    obsm: 'design_matrix', 'size_factors', 'replaceable'
    varm: 'non_zero', '_MoM_dispersions', 'genewise_dispersions', '_genewise_converged', '_normed_means', 'fitted_dispersions', 'MAP_dispersions', '_MAP_converged', 'dispersions', '_outlier_genes', 'LFC', '_LFC_converged', 'replaced', 'refitted'
    layers: 'normed_counts', '_mu_hat', '_mu_LFC', '_hat_diagonals', 'cooks'


In [9]:
print(dds.varm["LFC"])

                                             intercept  \
Sample                                                   
lcl|CACVAH010000001.1_cds_CAA6683422.1_1      5.646561   
lcl|CACVAH010000001.1_cds_CAA6683423.1_2      8.421647   
lcl|CACVAH010000001.1_cds_CAA6683424.1_3      7.843785   
lcl|CACVAH010000001.1_cds_CAA6683425.1_4      6.699215   
lcl|CACVAH010000001.1_cds_CAA6683426.1_5      6.453300   
...                                                ...   
lcl|CACVAH010000095.1_cds_CAA6687588.1_4089        NaN   
lcl|CACVAH010000096.1_cds_CAA6687590.1_4090        NaN   
lcl|CACVAH010000097.1_cds_CAA6687591.1_4091        NaN   
lcl|CACVAH010000098.1_cds_CAA6687592.1_4092        NaN   
lcl|CACVAH010000099.1_cds_CAA6687593.1_4093        NaN   

                                             Growth substrate_Mannose (Sigma-Aldrich M6020)_vs_Crude fucoidan from F. vesiculosus  (Sigma-Aldrich F5631)  \
Sample                                                                                   

In [38]:
# so in python you need to do the stdats externally not a part of general 
stat_res = DeseqStats(dds)
stat_res_Y_vs_X = DeseqStats(dds, contrast=["Growth substrate", "iota-carrageenan (Sigma-Aldrich C1138)", "Mannose (Sigma-Aldrich M6020)"])
stat_res_Y_vs_X.summary()

Running Wald tests...


Log2 fold change & Wald test p-value: Growth substrate iota-carrageenan (Sigma-Aldrich C1138) vs Mannose (Sigma-Aldrich M6020)
                                                baseMean  log2FoldChange  \
Sample                                                                     
lcl|CACVAH010000001.1_cds_CAA6683422.1_1      260.700796        0.111035   
lcl|CACVAH010000001.1_cds_CAA6683423.1_2     3986.627809       -1.183727   
lcl|CACVAH010000001.1_cds_CAA6683424.1_3     2255.379509       -1.036094   
lcl|CACVAH010000001.1_cds_CAA6683425.1_4      831.007130       -1.441429   
lcl|CACVAH010000001.1_cds_CAA6683426.1_5      638.966532       -1.517562   
...                                                  ...             ...   
lcl|CACVAH010000095.1_cds_CAA6687588.1_4089     0.000000             NaN   
lcl|CACVAH010000096.1_cds_CAA6687590.1_4090     0.000000             NaN   
lcl|CACVAH010000097.1_cds_CAA6687591.1_4091     0.000000             NaN   
lcl|CACVAH010000098.1_cds_CAA6687592.

... done in 2.41 seconds.



In [39]:
res2 = stat_res_Y_vs_X.results_df


In [49]:
res2.to_csv("")

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
Sample,,,,,,
lcl|CACVAH010000001.1_cds_CAA6683422.1_1,260.700796,0.111035,0.445747,0.249098,0.803285,0.999186
lcl|CACVAH010000001.1_cds_CAA6683423.1_2,3986.627809,-1.183727,0.579921,-2.041186,0.041232,0.838322
lcl|CACVAH010000001.1_cds_CAA6683424.1_3,2255.379509,-1.036094,0.502836,-2.060501,0.039351,0.832549
lcl|CACVAH010000001.1_cds_CAA6683425.1_4,831.007130,-1.441429,0.508037,-2.837251,0.004550,0.173687
lcl|CACVAH010000001.1_cds_CAA6683426.1_5,638.966532,-1.517562,0.721758,-2.102592,0.035501,0.793633
...,...,...,...,...,...,...
lcl|CACVAH010000095.1_cds_CAA6687588.1_4089,0.000000,NaN,NaN,NaN,NaN,NaN
lcl|CACVAH010000096.1_cds_CAA6687590.1_4090,0.000000,NaN,NaN,NaN,NaN,NaN
lcl|CACVAH010000097.1_cds_CAA6687591.1_4091,0.000000,NaN,NaN,NaN,NaN,NaN


# Lets loop this shit 

In [41]:
metadata

,Strain,Growth substrate,Timepoint [h],Sample accession,Identifier,Treatment
Sample,,,,,,
ERR3606203,Lentimonas' sp. CC6,iota-carrageenan (Sigma-Aldrich C1138),40,SAMEA6101969,LCC6_iota_carrageenan_40h,iota_carrageenan_40h
ERR3606204,Lentimonas' sp. CC6,iota-carrageenan (Sigma-Aldrich C1138),20,SAMEA6101970,LCC6_iota_carrageenan_20h,iota_carrageenan_20h
ERR3606205,Lentimonas' sp. CC6,Pure fucoidan from F. vesiculosus (Sigma-Aldri...,20,SAMEA6101971,LCC6_pure_fucoidan_20h,Pure_fucoidan_20h
ERR3606206,Lentimonas' sp. CC6,Pure fucoidan from F. vesiculosus (Sigma-Aldri...,40,SAMEA6101972,LCC6_pure_fucoidan_40h,Pure_fucoidan_40h
ERR3606207,Lentimonas' sp. CC6,Crude fucoidan from F. vesiculosus (Sigma-Ald...,40,SAMEA6101973,LCC6_crude_fucoidan_40h,Crude_fucoidan_40h
ERR3606208,Lentimonas' sp. CC6,Crude fucoidan from F. vesiculosus (Sigma-Ald...,20,SAMEA6101974,LCC6_crude_fucoidan_20h,Crude_fucoidan_20h
ERR3606209,Lentimonas' sp. CC6,Mannose (Sigma-Aldrich M6020),40,SAMEA6101975,LCC6_mannose_40h,Mannose_40h
ERR3606210,Lentimonas' sp. CC6,Mannose (Sigma-Aldrich M6020),20,SAMEA6101976,LCC6_mannose_20h,Mannose_20h


In [43]:
# make a list of unique values in column 
unique_values = metadata['Growth substrate'].unique().tolist()
unique_values

['iota-carrageenan (Sigma-Aldrich C1138)',
 'Pure fucoidan from F. vesiculosus (Sigma-Aldrich F8190)',
 'Crude fucoidan from F. vesiculosus  (Sigma-Aldrich F5631)',
 'Mannose (Sigma-Aldrich M6020)']

In [53]:
# Assuming your unique values are stored in a list called 'unique_values'
# Create an empty dictionary to store the results
results = {}

# Iterate over the unique values using a for loop
for i, value in enumerate(unique_values, 1):
    # Assuming 'stat_res_Y_vs_X' is your object of interest
    # Assuming 'DeseqStats' returns some result, replace this with your actual function call
    stat_d = DeseqStats(dds, contrast=['Growth substrate', value, 'Mannose (Sigma-Aldrich M6020)'])
    res2 = stat_d.results_df

AttributeError: 'DeseqStats' object has no attribute 'results_df'